### Version 1 - High Level Overview

In [12]:
from image_prep import get_img_df, train_val_test_split
from image_gen import get_gen_from_df

In [ ]:
fc_path = "data/Food_Classification/"

# get df
df_fc = get_img_df(fc_path)

num_classes = len(df_fc['label'].unique())
num_classes

In [14]:
# split dfs
train, validate, test = train_val_test_split(df_fc, test_size=0.2, val_size=0.2)

In [ ]:
train_gen = get_gen_from_df(train)
validate_gen = get_gen_from_df(validate, train=False)

The following model is probably not the final model we want to use but just needed something to test and end to end flow.  

This particular architecture was borrowed from the Jupyter notebook found: https://www.kaggle.com/code/varsha300/transferlearning


In [16]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Constants
IMG_SIZE = (224, 224)  # VGG16 default image size

# model

base_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
# Freeze base model layers and unfreeze the last 4 layers
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Model architecture
x = base_model.output
x = Flatten()(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train model
history = model.fit(train_gen, validation_data=validate_gen, epochs=1, steps_per_epoch=train_gen.num_batches)


In [ ]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(validate_gen)
print(f'Validation accuracy: {val_accuracy * 100:.2f}%')

### Version 2 - Find Best Model

In [1]:
import cv_models 

%load_ext autoreload
%autoreload 2

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4060 Laptop GPU, compute capability 8.9


In [ ]:
"""
from image_prep import get_split_data
df_train, df_validate, df_test = get_split_data()
train_gen, validate_gen = cv_models.get_training_data(df_train, df_validate, batch_size=32)

type(train_gen)
"""


In [2]:
"""
Best Accuracy: 0.13067729771137238
Best validation set: Model_Xception__Dropout_0.3__HiddenSize_200

Validation accuracy: 13.78%
Best Accuracy: 0.1378486007452011
Best validation set: Model_Xception__Dropout_0.2__HiddenSize_500
"""

# store results in each run in case of exception
for best_acc, best_model_name, best_dropout, best_hidden, all_results in cv_models.find_best_model(5, batch_size=16):
    continue



**********
Training: Model_Xception__Dropout_0.2__HiddenSize_500**********

Found 3760 validated image filenames belonging to 20 classes.
Found 1255 validated image filenames belonging to 20 classes.


c:\Users\Jon\miniconda3\envs\Tasty_AI\lib\site-packages\keras\preprocessing\image.py:1139: UserWarning: Found 2 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(


Epoch 1/5
235/235 [==============================] - 120s 496ms/step - loss: 8.1636 - accuracy: 0.0684 - val_loss: 7.9942 - val_accuracy: 0.1162
Epoch 2/5
235/235 [==============================] - 117s 495ms/step - loss: 8.1433 - accuracy: 0.0830 - val_loss: 8.5265 - val_accuracy: 0.1010
Epoch 3/5
235/235 [==============================] - 118s 503ms/step - loss: 8.0445 - accuracy: 0.0668 - val_loss: 8.8510 - val_accuracy: 0.0441
Epoch 4/5
235/235 [==============================] - 118s 502ms/step - loss: 7.8771 - accuracy: 0.0564 - val_loss: 8.3273 - val_accuracy: 0.0585
Epoch 5/5
79/79 [==============================] - 27s 347ms/step - loss: 7.8826 - accuracy: 0.0574
Validation accuracy: 5.74%
Best Accuracy: 0.057370517402887344
Best validation set: Model_Xception__Dropout_0.2__HiddenSize_500
**********
Training: Model_Xception__Dropout_0.2__HiddenSize_600**********

Found 3760 validated image filenames belonging to 20 classes.
Found 1255 validated image filenames belonging to 20 c

c:\Users\Jon\miniconda3\envs\Tasty_AI\lib\site-packages\keras\preprocessing\image.py:1139: UserWarning: Found 2 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(


Epoch 1/5
235/235 [==============================] - 123s 514ms/step - loss: 7.3291 - accuracy: 0.0362 - val_loss: 8.0979 - val_accuracy: 0.0272
Epoch 2/5
235/235 [==============================] - 119s 508ms/step - loss: 6.9270 - accuracy: 0.0346 - val_loss: 7.5158 - val_accuracy: 0.0481
Epoch 3/5
235/235 [==============================] - 120s 511ms/step - loss: 7.6604 - accuracy: 0.0407 - val_loss: 8.6931 - val_accuracy: 0.0665
Epoch 4/5
235/235 [==============================] - 121s 512ms/step - loss: 7.7683 - accuracy: 0.0330 - val_loss: 8.0285 - val_accuracy: 0.0481
Epoch 5/5
79/79 [==============================] - 27s 346ms/step - loss: 7.4877 - accuracy: 0.0510
Validation accuracy: 5.10%
**********
Training: Model_Xception__Dropout_0.2__HiddenSize_800**********

Found 3760 validated image filenames belonging to 20 classes.
Found 1255 validated image filenames belonging to 20 classes.


c:\Users\Jon\miniconda3\envs\Tasty_AI\lib\site-packages\keras\preprocessing\image.py:1139: UserWarning: Found 2 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(


Epoch 1/5
235/235 [==============================] - 124s 520ms/step - loss: 7.7013 - accuracy: 0.0258 - val_loss: 6.8134 - val_accuracy: 0.0160
Epoch 2/5
235/235 [==============================] - 121s 515ms/step - loss: 8.0329 - accuracy: 0.0314 - val_loss: 7.9751 - val_accuracy: 0.0264
Epoch 3/5
235/235 [==============================] - 121s 512ms/step - loss: 8.0967 - accuracy: 0.0410 - val_loss: 8.2384 - val_accuracy: 0.0521
Epoch 4/5
235/235 [==============================] - 121s 516ms/step - loss: 8.3784 - accuracy: 0.0407 - val_loss: 8.3242 - val_accuracy: 0.0817
Epoch 5/5
79/79 [==============================] - 27s 343ms/step - loss: 8.1370 - accuracy: 0.1068
Validation accuracy: 10.68%
Best Accuracy: 0.10677290707826614
Best validation set: Model_Xception__Dropout_0.2__HiddenSize_800
**********
Training: Model_Xception__Dropout_0.2__HiddenSize_1000**********

Found 3760 validated image filenames belonging to 20 classes.
Found 1255 validated image filenames belonging to 20 

c:\Users\Jon\miniconda3\envs\Tasty_AI\lib\site-packages\keras\preprocessing\image.py:1139: UserWarning: Found 2 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(


Epoch 1/5


ResourceExhaustedError: Graph execution error:

Detected at node 'tasty_model_3/xception/block8_sepconv3/separable_conv2d/Cast_1' defined at (most recent call last):
    File "c:\Users\Jon\miniconda3\envs\Tasty_AI\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Jon\miniconda3\envs\Tasty_AI\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\Jon\miniconda3\envs\Tasty_AI\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\Jon\miniconda3\envs\Tasty_AI\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\Jon\miniconda3\envs\Tasty_AI\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\Jon\miniconda3\envs\Tasty_AI\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_for

In [ ]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("TensorFlow is using GPU")
    for gpu in gpus:
        print("GPU:", gpu.name)
else:
    print("TensorFlow is not using GPU")